## Servicer Viability - Service (Chain) Growth

- **Goal**: to understand the impact on servicer profitability of a change in the number of services available
- **System Goals Targeted**:
  - Economic Viability
  - Long-Term Value Generation
  - Accessibility
- **Design**: create a stochastic model of servicer entry, conditional upon 1) the arrival of a new service (chain) into the ecosystem, 2) the response of applications to the arrival of a new service, and 3) the resulting profitability of a potential servicer entering the ecosystem
- **Testing mechanism**:
  - There is an arrival of a new service which initially has no servicers attached
  - There is relay demand in response to this new service, potentially with some delay
  - Servicers already in the ecosystem decide whether or not to relay to/from the new service
  - If a servicer decides to connect to the new service, it may _disconnect_ from its lowest-profit service (no unstaking penalty if moving from one service to another)
  - Potential servicers decide whether or not to enter the ecosystem
  - Different block reward and fee structures are tested against the above
- **KPIs**:
  - Servicer NPV (KPI-1)
  - Gateway NPV (KPI-3)
  - POKT Circulating Supply (KPI-4)
  - POKT Available Supply (KPI-5)
  - Mint/burn rate (KPI-D)
  - DAO Value Capture (KPI-10)
  - Servicer Capital Costs (KPI-14)

## Protocol Parameters to be swept

1. `RelaysToTokensMultiplier` (`RTTM`)
2. `GatewayFeePerRelay` (`GFPR`)
3. `ApplicationFeePerRelay` (`AFPR`, assumed different from `GFPR`)
4. `GatewayMinimumStake` (`GMS`)
5. `ApplicationMinimumStake` (`AMS`)
6. `DAOAllocation` (`DAL`)
7. `ValidatorAllocation` (`VAL`) 
8. `ServicerAllocation` (derived parameter, defined as 1 - `DAOAllocation` - `ValidatorAllocation`)

## Exogenous Parameters to be swept
1. Probability of a servicer attaching to a new service in the timestep the new service arrives: $p_c \in (0,1)$
2. Probability of a servicer detaching from the lowest-revenue existing service they are attached to: $p_d \in (0,1)$
3. Probability that a servicer joins the network in a given timestep: $p_s \in (0,1)$, where
$p_s = \bar p_s$ if **average servicer revenue expectations are positive for this timestep**, and $p_s = \underline p_s$ otherwise, with $\bar p_s \geq \underline p_s$.

## Exogenous Distributions that are fixed for this scenario

1. The probability distribution of relays requested by an application per day: $F^a(r; \pi) \in \mathcal{F}$. Probability distribution (within the set of continuous probability distributions with bounded support $\mathcal{F}$) of the number of relays per application per day (a random variable). This distribution is parameterized by a finite number of parameters $\pi$, $|\pi| < \infty$. Example parameters could be e.g. moments of the distribution.

    **For this scenario, the distribution is located in the function** `submit_relay_requests_ba_gamma()` [within model/boundary_actions/application_ba.py](https://github.com/BlockScience/PocketSimulationModel/blob/main/model/boundary_actions/application.py#L116) 


## Simulation Initialization

1. Initialize a set $N_s$ of servicers
2. Initialize a set $N_c$ of services (chains)
3. Initialize a set $N_a$ of applications
4. Initialize a set $N_g$ of gateways
5. Initialize the network of applications - gateways - services - servicers
6. Initialize the number of timesteps $T$


## Simulation routine
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' ecosystem profit/loss to servicers from services

In timestep $T/2 + 1$:
1. Instantiate a completely new service $C$, update $N_c$;
2. For each servicer in $N_s$:
  - **Attach** to $C$ with probability $p_c$;
  - **Detach** from lowest profitable service $\underline C$ with probability $p_d$
3. For each application in $N_a$:
   - Draw a relay demand realization $R^C$ from $F^a(r;\pi)$ for the new service $C$

**Simulation then runs for the remaining $T/2$ timesteps, with the network including the new service, and with a new servicer added probabilistically each timestep according to the value of $p_s$.**


Parameter Sweep Values (11 parameters in total)

|  Name | Sweep Values | Units |
| --- | ---| ---|
| `RTTM` | (100, 150, 200) | uPOKT/relay |
| `GFPR` | (20, 25, 30) | uPOKT/relay |
| `AFPR` | (20, 25, 30) | uPOKT/relay |
| `GMS`  | (1e5,1.5e5,2e5) | POKT |
| `AMS`  | (1e2, 1e3, 1e4) ? | POKT |
| `DAL`  | (0.05, 0.10, 0.15) | % |
| `VAL`  | (0.01, 0.05, 0.10) | % |
| $p_c$  | (0.1, 0.5, 0.9) | prob |
| $p_d$  | (0.01, 0.05, 0.1) | prob |
| $\bar p_s$  | (0.1, 0.5, 0.9) | prob |
| $\underline p_s$  | (0.01, 0.05, 0.1) | prob |

**Total number of parameter constellations**: $3^{11} = 177,147$

**Total number of Monte Carlo runs per constellation**: ? 30

**Total number of experiments**: 5.31441 million